<a href="https://colab.research.google.com/github/mdzikrim/Hands-on_DL/blob/main/Chapter_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
import numpy as np

# Simulasi: sequence (batch, time steps, notes)
# Format: batch of 4-note steps, note IDs 1-88, 0=no note
sequence = np.random.randint(0, 89, size=(1000, 64, 4))
X = sequence[:, :-1, :]  # Input
y = sequence[:, 1:, :]   # Target (next step)

# Convert to one-hot
vocab_size = 89
X_oh = tf.one_hot(X, vocab_size)
y_oh = tf.one_hot(y, vocab_size)

# Reshape y_oh to match model output shape
y_oh = tf.reshape(y_oh, shape=(-1, y_oh.shape[1], y_oh.shape[2] * y_oh.shape[3]))

In [8]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=[None, 4, vocab_size]),
    layers.TimeDistributed(layers.Flatten()),
    layers.LSTM(128, return_sequences=True),
    layers.TimeDistributed(layers.Dense(128, activation='relu')),
    layers.TimeDistributed(layers.Dense(4 * vocab_size, activation="softmax"))
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_6              │ (None, None, 356)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, None, 128)      │       248,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, None, 128)      │        16,512 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, None, 356)      │        45,924 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 310,756 (1.19 MB)

 Trainable params: 310,756 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.fit(X_oh, y_oh, batch_size=32, epochs=5)


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 235ms/step - accuracy: 0.0019 - loss: 23.5007
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 215ms/step - accuracy: 0.0033 - loss: 23.4927
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 200ms/step - accuracy: 0.0040 - loss: 23.4819
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 217ms/step - accuracy: 0.0060 - loss: 23.4648
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 218ms/step - accuracy: 0.0069 - loss: 23.4348


In [11]:
# Ambil awalan dari sequence real
input_seq = X_oh[0:1]  # (1, timesteps, 4, vocab)

# Prediksi 20 langkah selanjutnya
generated = []

for _ in range(20):
    preds = model.predict(input_seq, verbose=0)
    # Reshape preds to (batch_size, timesteps, 4, vocab_size) before argmax
    preds_reshaped = tf.reshape(preds, shape=(-1, preds.shape[1], 4, vocab_size))
    next_notes = tf.argmax(preds_reshaped[:, -1], axis=-1).numpy() # Should have shape (1, 4)

    # One-hot encode the next notes and reshape for concatenation
    next_notes_oh = tf.one_hot(next_notes, vocab_size) # Shape (1, 4, vocab_size)
    next_notes_oh_expanded = tf.expand_dims(next_notes_oh, axis=1) # Shape (1, 1, 4, vocab_size)

    input_seq = tf.concat([input_seq, next_notes_oh_expanded], axis=1)
    generated.append(next_notes)

print("Generated sequence (note ids):")
print(np.concatenate(generated, axis=0))

Generated sequence (note ids):
[[31  0  3 51]
 [57  0  4 51]
 [57  0  4 22]
 [57  0 82 22]
 [57  0 36 36]
 [57  0 82 36]
 [57  0 38 36]
 [57 39 38 36]
 [57 39 38 36]
 [34 39 38 36]
 [34 39 38 36]
 [38 39 79 36]
 [34 39 79 36]
 [34 39 79 36]
 [34 61 79 36]
 [34 61 79 36]
 [34 61 79 36]
 [34 72 79 36]
 [34 72 79 36]
 [38 72 79 36]]
